# A work in progress notebook for entity linking

(Submission is currently in progress, looks like we have to lowercase all queries before linking the entities, currently discussing this with Marcel)

In [ ]:
# Only needed in Colab, in codespaces everything is already installed.
!pip3 install python-terrier tira ir-datasets

In [ ]:
import pyterrier as pt
from tqdm import tqdm
import pandas as pd

if not pt.started():
  pt.init()

from tira.rest_api_client import Client
tira = Client()

terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



# Example Evaluation on a single dataset

In [ ]:
dataset = pt.get_dataset("irds:disks45/nocr/trec-robust-2004")
topics = dataset.get_topics(variant='title')
topics.head(3)

,qid,query
0,301,international organized crime
1,302,poliomyelitis and post polio
2,303,hubble telescope achievements


In [ ]:
query_entity_linking = tira.pt.transform_queries('ir-benchmarks/marcel-gohsen/wood-block', dataset)
query_entity_linking(topics).head(3)

,qid,query,original_query,entities
0,301,International Organized Crime,"{'query_id': '301', 'title': 'International Or...","[{'begin': 14, 'end': 29, 'mention': 'organize..."
1,302,Poliomyelitis and Post-Polio,"{'query_id': '302', 'title': 'Poliomyelitis an...","[{'begin': 0, 'end': 5, 'mention': 'polio', 'u..."
2,303,Hubble Telescope Achievements,"{'query_id': '303', 'title': 'Hubble Telescope...","[{'begin': 7, 'end': 16, 'mention': 'telescope..."


In [ ]:
query_entity_linking(topics).iloc[0].to_dict()

{'qid': '301',
 'query': 'International Organized Crime',
 'original_query': {'query_id': '301',
  'title': 'International Organized Crime',
  'description': 'Identify organizations that participate in international criminal\nactivity, the activity, and, if possible, collaborating organizations\nand the countries involved.',
  'narrative': 'A relevant document must as a minimum identify the organization and the\ntype of illegal activity (e.g., Columbian cartel exporting cocaine).\nVague references to international drug trade without identification of\nthe organization(s) involved would not be relevant.'},
 'entities': [{'begin': 14,
   'end': 29,
   'mention': 'organized crime',
   'url': 'https://en.wikipedia.org/wiki/Organized_crime',
   'score': 0.97188995215311},
  {'begin': 0,
   'end': 29,
   'mention': 'international organized crime',
   'url': 'https://en.wikipedia.org/wiki/Transnational_organized_crime',
   'score': 0.33333333333333304},
  {'begin': 0,
   'end': 13,
   'mentio

In [ ]:
query_entity_linking(topics).iloc[247].to_dict()

{'qid': '698',
 'query': 'literacy rates Africa',
 'original_query': {'query_id': '698',
  'title': 'literacy rates Africa',
  'description': 'What are literacy rates in African countries?',
  'narrative': 'A relevant document will contain information about the\nliteracy rate in an African country.\nGeneral education levels that do not specifically include literacy rates\nare not relevant.'},
 'entities': [{'begin': 15,
   'end': 21,
   'mention': 'africa',
   'url': 'https://en.wikipedia.org/wiki/Africa',
   'score': 0.904072542157174},
  {'begin': 0,
   'end': 8,
   'mention': 'literacy',
   'url': 'https://en.wikipedia.org/wiki/Literacy',
   'score': 0.710061993323795},
  {'begin': 0,
   'end': 14,
   'mention': 'literacy rates',
   'url': 'https://en.wikipedia.org/wiki/List_of_countries_by_literacy_rate',
   'score': 0.09090909090909001},
  {'begin': 15,
   'end': 21,
   'mention': 'africa',
   'url': 'https://en.wikipedia.org/wiki/Africa_(Roman_province)',
   'score': 0.0167435571

# Example Evaluation on All Datasets

In [ ]:
ir_datasets = ['clueweb09/en/trec-web-2009', 'clueweb09/en/trec-web-2010', 'clueweb09/en/trec-web-2011',
               'clueweb09/en/trec-web-2012', 'clueweb12/trec-web-2013', 'clueweb12/trec-web-2014',
               'disks45/nocr/trec-robust-2004', 'antique/test', 'argsme/2020-04-01/touche-2020-task-1',
               'argsme/2020-04-01/touche-2021-task-1', 'clueweb12/touche-2020-task-2', 'clueweb12/touche-2021-task-2',
               'cord19/fulltext/trec-covid', 'cranfield', 'disks45/nocr/trec7', 'disks45/nocr/trec8',
               'gov/trec-web-2002', 'gov/trec-web-2003', 'gov/trec-web-2004', 'gov2/trec-tb-2004',
               'gov2/trec-tb-2005', 'gov2/trec-tb-2006', 'medline/2004/trec-genomics-2004',
               'medline/2004/trec-genomics-2005', 'medline/2017/trec-pm-2017', 'medline/2017/trec-pm-2018',
               'msmarco-passage/trec-dl-2019/judged', 'msmarco-passage/trec-dl-2020/judged',
               'nfcorpus/test', 'vaswani', 'wapo/v2/trec-core-2018']
df_all = []

for dataset in tqdm(ir_datasets):
  pt_dataset = pt.get_dataset(f"irds:{dataset}")
  topics = pt_dataset.get_topics()
  query_entity_linking = tira.pt.transform_queries('ir-benchmarks/marcel-gohsen/wood-block', dataset)
  df = query_entity_linking(topics)
  df['dataset'] = dataset

  df_all += [df]

df_all = pd.concat(df_all)
df_all

NameError: name 'tqdm' is not defined

In [ ]:
df_all['entity_count'] = df_all['entities'].apply(lambda i: len(i))
df_all.sort_values('entity_count')

,qid,query,description,type,subtopics,original_query,entities,dataset,title,narrative,need,context,disease,gene,demographic,other,all,entity_count
2,53,Should blood donations be financially compensa...,NaN,NaN,NaN,"{'query_id': '53', 'title': 'Should blood dona...",[],argsme/2020-04-01/touche-2021-task-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
39,40,michworks,Find information on getting a job in Michigan....,faceted,"((1, \n Take me to the michworks Michigan T...","{'query_id': '40', 'query': 'michworks', 'desc...",[],clueweb09/en/trec-web-2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
96,2180086,what are mormens?,NaN,NaN,NaN,"{'query_id': '2180086', 'text': 'what are morm...",[],antique/test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
44,145,vines for shade,\n information on vines that can be grown i...,ambiguous,"((1, \n information on vines that can be gr...","{'query_id': '145', 'query': 'vines for shade'...",[],clueweb09/en/trec-web-2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
46,47,indexed annuity,I'm looking for information about indexed annu...,faceted,"((1, \n What is an indexed annuity? What a...","{'query_id': '47', 'query': 'indexed annuity',...",[],clueweb09/en/trec-web-2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,103,how far around a cylinder and under what condi...,NaN,NaN,NaN,"{'query_id': '103', 'text': 'how far around a ...","[{'begin': 123, 'end': 129, 'mention': 'linear...",cranfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216
153,2452795,How can I heat the water within a 150 gal tank...,NaN,NaN,NaN,"{'query_id': '2452795', 'text': 'How can I hea...","[{'begin': 19, 'end': 24, 'mention': 'water', ...",antique/test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,221
136,206,have any analytical studies been conducted on ...,NaN,NaN,NaN,"{'query_id': '206', 'text': 'have any analytic...","[{'begin': 33, 'end': 42, 'mention': 'conducte...",cranfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222
91,140,given complete freedom in the design of an air...,NaN,NaN,NaN,"{'query_id': '140', 'text': 'given complete fr...","[{'begin': 43, 'end': 51, 'mention': 'airplane...",cranfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250


In [ ]:
df_all.sort_values('entity_count').head(100)

,qid,query,description,type,subtopics,original_query,entities,dataset,title,narrative,need,context,disease,gene,demographic,other,all,entity_count
2,53,Should blood donations be financially compensa...,NaN,NaN,NaN,"{'query_id': '53', 'title': 'Should blood dona...",[],argsme/2020-04-01/touche-2021-task-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
39,40,michworks,Find information on getting a job in Michigan....,faceted,"((1, \n Take me to the michworks Michigan T...","{'query_id': '40', 'query': 'michworks', 'desc...",[],clueweb09/en/trec-web-2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
96,2180086,what are mormens?,NaN,NaN,NaN,"{'query_id': '2180086', 'text': 'what are morm...",[],antique/test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
44,145,vines for shade,\n information on vines that can be grown i...,ambiguous,"((1, \n information on vines that can be gr...","{'query_id': '145', 'query': 'vines for shade'...",[],clueweb09/en/trec-web-2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
46,47,indexed annuity,I'm looking for information about indexed annu...,faceted,"((1, \n What is an indexed annuity? What a...","{'query_id': '47', 'query': 'indexed annuity',...",[],clueweb09/en/trec-web-2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,317,Unsolicited Faxes,Have regulations been passed by the FCC bannin...,NaN,NaN,"{'query_id': '317', 'title': 'Unsolicited Faxe...","[{'begin': 12, 'end': 17, 'mention': 'faxes', ...",disks45/nocr/trec-robust-2004,Unsolicited Faxes,Relevant documents will provide information on...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
217,218,MedWatch,NaN,NaN,NaN,"{'query_id': '218', 'text': 'MedWatch'}","[{'begin': 0, 'end': 8, 'mention': 'medwatch',...",gov/trec-web-2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
283,PLAIN-593,apnea,NaN,NaN,NaN,"{'query_id': 'PLAIN-593', 'title': 'apnea', 'a...","[{'begin': 0, 'end': 5, 'mention': 'apnea', 'u...",nfcorpus/test,apnea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"apnea - - infants , milk , sids , dairy , crib...",1
20,321,Women in Parliaments,Pertinent documents will reflect the fact that...,NaN,NaN,"{'query_id': '321', 'title': 'Women in Parliam...","[{'begin': 9, 'end': 20, 'mention': 'parliamen...",disks45/nocr/trec-robust-2004,Women in Parliaments,Pertinent documents relating to this issue wil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
